In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import pandas as pd
# Import the backtrader platform
import backtrader as bt
import backtrader.indicators as btind
#import pandas as pd
# to avoid plot disappear everytime rerun the code
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#plt.figure(figsize=(15,5))

In [3]:
# create a data feed and set formet
#data_sentiment=bt.feeds.GenericCSVData(dataname="/Users/tyllersun/Desktop/大一/data_science/bond.csv")
#/Users/tyllersun/Desktop/data0.csv
#data_1 = bt.feeds.GenericCSVData(dataname='/Users/tyllersun/Desktop/data0_test.csv',
data_1 = bt.feeds.GenericCSVData(dataname='/Users/tyllersun/Desktop/data_XLX.csv',
fromdate=datetime.datetime(2015, 5, 1), # startdate
todate=datetime.datetime(2019, 1, 1), # enddate
                                     
dtformat=('%Y-%m-%d'),tmformat=('%H.%M.%S'),
    date=0, open=1, close=2, high=3, low=4,
    volume=7, openinterest=8, positive_std=-1, negative_std=-1 ,code=-1,
    reverse=False)
#print("finish")
#print(data_1.positive_std)

In [4]:
# Create a Stratey #Buy if price continuelly fall for 3d & Sell every 5d
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
                
        # To keep track of pending orders #掛單
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None      
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))    
    
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):  # 每過5天 -> sell!
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
               

In [5]:
class sentiment_strategy(bt.Strategy):
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None      
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))    
    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        #print("s")
        #high
        #print(self.datas[0].volume)
        self.data_positive = self.datas[0].volume
        self.data_negative = self.datas[0].openinterest
        self.dataclose = self.datas[0].close
        #print("finish_init")
           
    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('sentiment, %.2f' % self.dataclose[0])
        #print("a")
        time=10
        total_pos=0
        total_neg=0
        for i in range(time):
            where=i-14
            total_pos=total_pos+self.data_positive[where]
            total_neg=total_neg+self.data_negative[where]
        total_pos_avg=total_pos/time
        total_neg_avg=total_neg/time
        #print("b")
        if  total_pos_avg > self.data_positive[0]*(2.3):
            #print("w")
            print(total_pos_avg)
            self.log('BUY CREATE, %.2f' % self.dataclose[0])
            self.buy(size=5)
        else: 
            if total_neg_avg > self.data_negative[0]*(2.3):
                #print("c")
                print(total_neg_avg)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.sell(size=5)
       
    """"   
        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()
        """

In [6]:
class RSI_Strategy(bt.Strategy):

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.buy(size=100)
        else:
            if self.rsi > 70:
                self.sell(size=100)

In [7]:
class MA_CrossOver(bt.Strategy):
    '''This is a long-only strategy which operates on a moving average cross
    Note:
      - Although the default
    Buy Logic:
      - No position is open on the data
      - The ``fast`` moving averagecrosses over the ``slow`` strategy to the
        upside.
    Sell Logic:
      - A position exists on the data
      - The ``fast`` moving average crosses over the ``slow`` strategy to the
        downside
    Order Execution Type:
      - Market
    '''
    alias = ('SMA_CrossOver',)

    params = (
        # period for the fast Moving Average
        ('fast', 10),
        # period for the slow moving average
        ('slow', 30),
        # moving average to use
        ('_movav', btind.MovAv.SMA)
    )

    def __init__(self):
        sma_fast = self.p._movav(period=self.p.fast)
        sma_slow = self.p._movav(period=self.p.slow)

        self.buysig = btind.CrossOver(sma_fast, sma_slow)

    def next(self):
        if self.position.size:
            if self.buysig < 0:
                self.sell()

        elif self.buysig > 0:
            self.buy()

In [8]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 10, 8  # that's default image size for this interactive session

pylab.rcParams['font.family'] = 'Segoe Script'
pylab.rcParams['font.sans-serif'] = ['Segoe Script']

#pylab.rcParams['font.family'] = 'sans-serif'
#pylab.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
#pylab.rcParams['font.serif'] = ['Bitstream Vera Sans']
pylab.rcParams["font.size"] = "10"

In [9]:
import pandas as pd
if __name__ == '__main__':
    #sentiment=pd.read_excel("/Users/tyllersun/Desktop/大一/data_science/bond.csv")
    cerebro = bt.Cerebro() # creat the "brain"

    # Add a strategy
    cerebro.addstrategy(sentiment_strategy)

    cerebro.adddata(data_1)
    
    cerebro.broker.setcash(100000.0) # the "manager"
    
    # Set the commission - 0.1% ... divide by 100 to remove the % #手續費
    cerebro.broker.setcommission(commission=0.001)
    
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run() # run over everything
    
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue()) 
    
    cerebro.plot() # plot the result

Starting Portfolio Value: 100000.00
0.3179444770532662
2015-05-01, SELL CREATE, 40.40
2015-05-04, SELL EXECUTED, Price: 40.47, Cost: -202.36, Comm 0.20
0.020385765739232897
2015-05-04, BUY CREATE, 40.62
2015-05-05, BUY EXECUTED, Price: 40.31, Cost: -202.36, Comm 0.20
2015-05-05, OPERATION PROFIT, GROSS 0.80, NET 0.39
0.07441475969302618
2015-05-05, BUY CREATE, 40.38
2015-05-06, BUY EXECUTED, Price: 39.94, Cost: 199.69, Comm 0.20
0.1151582260547734
2015-05-06, BUY CREATE, 40.01
2015-05-07, BUY EXECUTED, Price: 39.52, Cost: 197.58, Comm 0.20
0.2218693789994058
2015-05-07, BUY CREATE, 39.88
2015-05-08, BUY EXECUTED, Price: 40.14, Cost: 200.72, Comm 0.20
0.3281813804088741
2015-05-08, SELL CREATE, 40.34
2015-05-11, SELL EXECUTED, Price: 40.28, Cost: 199.33, Comm 0.20
0.11831497050338388
2015-05-11, BUY CREATE, 40.33
2015-05-12, BUY EXECUTED, Price: 39.89, Cost: 199.46, Comm 0.20
0.16160825422248432
2015-05-12, BUY CREATE, 40.07
2015-05-13, BUY EXECUTED, Price: 40.06, Cost: 200.30, Comm 0.2

<IPython.core.display.Javascript object>